In [2]:
# !pip install -q "arize-phoenix>=8.0.0" datasets

In [ ]:
pip install -U dspy

In [3]:
'''
import os
from getpass import getpass

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
if not os.environ.get("PHOENIX_CLIENT_HEADERS"):
    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
'''

'\nimport os\nfrom getpass import getpass\n\nos.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"\nif not os.environ.get("PHOENIX_CLIENT_HEADERS"):\n    os.environ["PHOENIX_CLIENT_HEADERS"] = "api_key=" + getpass("Enter your Phoenix API key: ")\n\nif not os.environ.get("OPENAI_API_KEY"):\n    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")\n'

In [5]:
pip install dspy

  Using cached dspy-2.6.10-py3-none-any.whl.metadata (7.3 kB)
Using cached dspy-2.6.10-py3-none-any.whl (260 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# imports

import openai
import dspy
import json
import re
import contextlib
import signal
from typing import *
import random
import os
from enum import Enum
import numpy as np
import tqdm
from yaml import safe_load
from dataclasses import dataclass, field
import requests
import textwrap

d:\anaconda\envs\mapcoder\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# formatting
def parse_code( response: str) -> str:
    if "```" not in response:
        return response

    code_pattern = r'```((.|\n)*?)```'
    if "```Python" in response:
        code_pattern = r'```Python((.|\n)*?)```'
    if "```Python3" in response:
        code_pattern = r'```Python3((.|\n)*?)```'
    if "```python" in response:
        code_pattern = r'```python((.|\n)*?)```'
    if "```python3" in response:
        code_pattern = r'```python3((.|\n)*?)```'
    if "```C" in response:
        code_pattern = r'```C((.|\n)*?)```'
    if "```c" in response:
        code_pattern = r'```c((.|\n)*?)```'
    if "```C++" in response:
        code_pattern = r'```C\+\+((.|\n)*?)```'
    if "```c++" in response:
        code_pattern = r'```c\+\+((.|\n)*?)```'
    if "```Java" in response:
        code_pattern = r'```Java((.|\n)*?)```'
    if "```java" in response:
        code_pattern = r'```java((.|\n)*?)```'
    if "```Node" in response:
        code_pattern = r'```Node((.|\n)*?)```'
    if "```node" in response:
        code_pattern = r'```node((.|\n)*?)```'
    if "```Rust" in response:
        code_pattern = r'```Rust((.|\n)*?)```'
    if "```rust" in response:
        code_pattern = r'```rust((.|\n)*?)```'
    if "```PHP" in response:
        code_pattern = r'```PHP((.|\n)*?)```'
    if "```php" in response:
        code_pattern = r'```php((.|\n)*?)```'
    if "```Go" in response:
        code_pattern = r'```Go((.|\n)*?)```'
    if "```go" in response:
        code_pattern = r'```go((.|\n)*?)```'
    if "```Ruby" in response:
        code_pattern = r'```Ruby((.|\n)*?)```'
    if "```ruby" in response:
        code_pattern = r'```ruby((.|\n)*?)```'
    if "```C#" in response:
        code_pattern = r'```C#((.|\n)*?)```'
    if "```c#" in response:
        code_pattern = r'```c#((.|\n)*?)```'
    if "```csharp" in response:
        code_pattern = r'```csharp((.|\n)*?)```'

    code_blocks = re.findall(code_pattern, response, re.DOTALL)

    if type(code_blocks[-1]) == tuple or type(code_blocks[-1]) == list:
        code_str = "\n".join(code_blocks[-1])
    elif type(code_blocks[-1]) == str:
        code_str = code_blocks[-1]
    else:
        code_str = response

    return code_str

In [ ]:
# api_key
openai_api_key = ""

In [9]:
# dspy configuration

lm = dspy.LM('openai/gpt-4o-mini', api_key = openai_api_key)
dspy.configure(lm = lm)

In [10]:
# OpenAi configuration

os.environ["OPENAI_API_KEY"] = openai_api_key

def gpt_chat(processed_input: List[Dict[str, str]], model="gpt-4o-mini", max_tokens=4096):
  """
  直接调用 OpenAI API 进行对话

  :param processed_input: List[Dict[str, str]] - 聊天输入，格式为 [{'role': 'system', 'content': 'Hello'}]
  :param model: str - 要使用的 OpenAI 模型，默认为 'gpt-4'
  :param max_tokens: int - 生成的最大 token 数量
  :return: (str, int, int) - 返回的文本、prompt token 计数、completion token 计数
  """
  response = openai.chat.completions.create(
      model=model,
      messages=processed_input,
      max_tokens=max_tokens
  )

  return response.choices[0].message.content, response.usage.prompt_tokens, response.usage.completion_tokens


In [11]:
# Executor utils
def timeout_handler(_, __):
    raise TimeoutError()

import os, json
def to_jsonl(dict_data, file_path):
    with open(file_path, 'a') as file:
        json_line = json.dumps(dict_data)
        file.write(json_line + os.linesep)

from threading import Thread
class PropagatingThread(Thread):
    def run(self):
        self.exc = None
        try:
            if hasattr(self, '_Thread__target'):
                # Thread uses name mangling prior to Python 3.
                self.ret = self._Thread__target(*self._Thread__args, **self._Thread__kwargs)
            else:
                self.ret = self._target(*self._args, **self._kwargs)
        except BaseException as e:
            self.exc = e

    def join(self, timeout=None):
        super(PropagatingThread, self).join(timeout)
        if self.exc:
            raise self.exc
        return self.ret


def function_with_timeout(func, args, timeout):
    result_container = []

    def wrapper():
        result_container.append(func(*args))

    thread = PropagatingThread(target=wrapper)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        raise TimeoutError()
    else:
        return result_container[0]

In [12]:
# func evaluate
def evaluate_io(
  sample_io: list[str],
  completion: str,
  timeout: int = 5,
  stop_early: bool = False,
):
  test_log = ""
  passed = True
  for io in sample_io:
    try:
      code = ("from typing import *\n" if "from typing import *" not in completion else "") + \
        completion + "\n" + io + "\n"
      function_with_timeout(
        exec,
        (code, globals()),
        timeout
      )
      test_log += f"passed in test case: {io}\n"
    except Exception as e:
      if stop_early:
        return False, f"failed in test case: {io}\n"
      passed = False
      test_log += f"failed in test case: {io}\n"

  return passed, test_log


def evaluate_io_et(
  sample_io: list[str],
  completion: str,
  timeout: int = 5,
  prompt: str = "",
):
  io = "\n".join(sample_io)
  try:
    code = ("from typing import *\n" if "from typing import *" not in completion else "") + \
      prompt + completion + "\n" + io + "\n"
    function_with_timeout(
      exec,
      (code, globals()),
      timeout
    )
    return True
  except Exception as e:
    return False


def evaluate_functional_correctness(
  problem: Dict,
  completion: str,
  timeout: int = 5,
  test_key: str = "test",
):
  # if problem["name"] == "mbpp_61_count_Substrings":
  #     pass
  try:
    code = ("from typing import *\n" if "from typing import *" not in completion else "") + \
      completion + "\n" + problem[test_key] + \
      "\n" + f"check({problem['entry_point']})"

    function_with_timeout(
      exec,
      (code, globals()),
      timeout
    )
    return "passed"
  except Exception as e:
    return f"failed: {e}"


def evaluate_functional_correctness2(
  problem: Dict,
  completion: str,
  timeout: float = 10,
) -> Dict:

  check_program = (
    # problem["prompt"] +
    "from typing import *\n" +
    completion + "\n" +
    problem["test"] + "\n" +
    f"check({problem['entry_point']})"
  )
  # print(check_program)

  try:
    exec(check_program)
    return "passed"
  except TimeoutException:
    return "timed out"
  except BaseException as e:
    return f"failed: {e}"


class TimeoutException(Exception):
  pass


In [13]:
def evaluate(
  item: dict,
  cur_imp: str,
  language: str,
):
  result = evaluate_functional_correctness(
    problem=item,
    completion=cur_imp
  )
  return result == "passed"

def humaneval_evaluate_sample_io(
  item: dict,
  cur_imp: str,
  language: str,
):

  return evaluate_io(
    sample_io=item["sample_io_prompt"],
    completion=cur_imp,
    )

In [14]:
class ExecOutcome(Enum):
    PASSED = "PASSED"  # code executes and output matches expected output
    WRONG_ANSWER = (
        "WRONG_ANSWER"  # code executes and output does NOT matches expected output
    )
    TIME_LIMIT_EXCEEDED = "TIME_LIMIT_EXCEEDED"  # code executes and didn't exit in time, output is ignored in this case
    RUNTIME_ERROR = "RUNTIME_ERROR"  # code failed to execute (crashed)
    COMPILATION_ERROR = "COMPILATION_ERROR"  # code failed to compile
    MEMORY_LIMIT_EXCEEDED = (
        "MEMORY_LIMIT_EXCEEDED"  # code exceeded memory limit during execution
    )

@dataclass
class ExtendedUnittest:
    input: str
    output: list[str] = field(default_factory=list)
    result: str | None = None
    exec_outcome: ExecOutcome | None = None

    def json(self):
        _json = self.__dict__
        if self.exec_outcome is not None:
            _json["exec_outcome"] = self.exec_outcome.name

        return _json

    @classmethod
    def from_json(cls, _json):
        return cls(
            input=_json.get("input", ""),
            output=_json.get("output", list()),
            result=_json.get("result", None),
            exec_outcome=_json.get("exec_outcome", None),
        )


class EmptyValueError(Exception):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


class EmptyUnittestError(EmptyValueError):
    pass


class EmptyLanguageError(EmptyValueError):
    pass


class EmptySourceCodeError(EmptyValueError):
    pass


class APICommunication:
    _session: requests.Session

    def __init__(self, server_url: str = "http://localhost:5000"):
        self._session = requests.Session()
        self.execute_code_url = f"{server_url}/api/execute_code"
        self.get_runtimes_url = f"{server_url}/api/all_runtimes"

    def __enter__(self):
        return self

    def __exit__(self, *args):
        self._session.close()

    def get_runtimes(self):
        return self._session.get(self.get_runtimes_url).json()

    def execute_code(
        self,
        language: str,
        source_code: str,
        unittests: list[dict],
        limits: dict | None,
        block_network: bool = True,
        stop_on_first_fail: bool = True,
        use_sanitizer: bool = False,
        compiler_program_name: str | None = None,
        compiler_flags: str | None = None,
        interpreter_cmd: str | None = None,
        interpreter_flags: str | None = None,
        sample_id: int | None = None,
        task_id: str | int | None = None,
    ) -> tuple[list[ExtendedUnittest], int | None, str | int | None]:
        if language is None:
            raise EmptyLanguageError

        if source_code is None:
            raise EmptySourceCodeError

        if unittests is None or len(unittests) == 0:
            raise EmptyUnittestError

        request_body = dict(
            language=language,
            source_code=source_code,
            unittests=unittests,
            limits=limits if isinstance(limits, dict) else dict(),
            compile_cmd=compiler_program_name,
            compile_flags=compiler_flags,
            execute_cmd=interpreter_cmd,
            execute_flags=interpreter_flags,
            block_network=block_network,
            stop_on_first_fail=stop_on_first_fail,
            use_sanitizer=use_sanitizer,
        )
        json_response = self._session.post(
            self.execute_code_url,
            json=request_body,
            headers={"Content-Type": "application/json"},
        ).json()

        if "data" not in json_response:
            return "error", sample_id, task_id

        return (
            json_response["data"],
            sample_id,
            task_id,
        )


In [22]:

LANGUAGE_MAPPING = {
    "Python": "Python 3",
    "Python3": "Python 3",
    "python": "Python 3",
    "C#": "C# 10",
    "NET-CORE": ".NET Core C#",
    # "Node": "Node.js",
    "Rust": "Rust",
    # "Java":"Java 17",
    "PHP": "PHP",
    "Go": "Go",
    "Ruby": "Ruby",
    "C++": "GNU C++17",
    "C": "GNU C"
}

colab_limits_by_lang_cfg_file = "/content/limits_by_lang.yaml"
local_limits_by_lang_cfg_file = r"C:\Users\31646\Desktop\LLM_Project\MapCoder\src\evaluations\limits_by_lang.yaml"

limits_by_lang_cfg_file = local_limits_by_lang_cfg_file

assert os.path.exists(
    limits_by_lang_cfg_file), "Need resource limit defaults for all runtimes, provide the path to default 'limits_by_lang.yaml' or to the modified one."

with open(limits_by_lang_cfg_file) as limit_cfg_rp:
    limits_by_lang = safe_load(limit_cfg_rp)


api_comm = APICommunication()


def contest_evaluate(
    generated_code: str,
    lang: str,
    id: int,
    tests: List[dict],
):
    assert lang in LANGUAGE_MAPPING, f"language must be inside the supported language list: {LANGUAGE_MAPPING.keys()}"

    results, _, _ = api_comm.execute_code(
        language=LANGUAGE_MAPPING[lang],
        source_code=generated_code,
        unittests=tests,
        limits=limits_by_lang[LANGUAGE_MAPPING[lang]],
        task_id=id,
    )

    if results == "error":
        return False

    passed = True
    for result in results:
        if result['exec_outcome'] != ExecOutcome.PASSED.value:
            passed = False
            break

    return passed


def contest_evaluate_public_tests(
    generated_code: str,
    lang: str,
    id: int,
    tests: List[dict],
):
    results, _, _ = api_comm.execute_code(
        language=LANGUAGE_MAPPING[lang],
        source_code=generated_code,
        unittests=tests,
        limits=limits_by_lang[LANGUAGE_MAPPING[lang]],
        task_id=id,
        stop_on_first_fail=False
    )

    passed = True
    passed_feedback = []
    failed_feedback = []

    idx = 0
    try:
        for idx, result in enumerate(results):
            output = str(result['result'])
            if len(output) > 500:
                output = output[:500] + "..."
            test_case = f"Input:\n{tests[idx]['input']}\nExpected Output:\n{tests[idx]['output'][0]}\nYour Output:\n{output}\n"
            if result['exec_outcome'] == ExecOutcome.PASSED.value:
                passed_feedback.append(test_case)
            if result['exec_outcome'] != ExecOutcome.PASSED.value:
                failed_feedback.append(test_case)
                passed = False
    except:
        passed = False
        test_cases = []
        for i in range(idx, len(tests)):
            test_case = f"Input:\n{tests[i]['input']}\nExpected Output:\n{tests[i]['output'][0]}\n"
            test_cases.append(test_case)

        failed_feedback.extend(test_cases)

    passed_feedback = '\n'.join(passed_feedback) if len(passed_feedback) > 0 else "No test cases passed."
    failed_feedback = '\n'.join(failed_feedback)
    feedback = f'## Tested passed:\n{passed_feedback}\n\n## Tests failed:\n{failed_feedback}'

    return passed, feedback


In [20]:
def coding_agent_prompt(algorithm_prompt, task, planning, sample_io_prompt, std_input_prompt):
  return [
    {
"role": "user",
"content":
f"""Given a competitive programming problem, generate python code to solve the problem.
{algorithm_prompt}
## Problem to be solved:
{task}
## Planning:
{planning}
{sample_io_prompt}
## Let's think step by step.
----------------
Important:
{std_input_prompt}
## Your response must contain only the python code to solve this problem.
Do not add extra explanation or words.
"""
  }
]

In [60]:
# Load HumanEval dataset
def read_jsonl(filename):
  """Reads a jsonl file and yields each line as a dictionary"""
  lines = []
  with open(filename, "r", encoding="utf-8") as file:
      for line in file:
          lines.append(json.loads(line))
  return lines

# Humaneval
from datasets import load_dataset

local_path = r"C:\Users\31646\Desktop\LLM_Project\MapCoder\data\HumanEval\HumanEvalWST.jsonl"
colab_path = "/content/HumanEvalWST.jsonl"

path = local_path

humaneval_ds = read_jsonl(path)
print(humaneval_ds[0].keys())

# select random 20% as a trial
trainset = humaneval_ds

dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'sample_io'])


In [61]:
def get_sample_io_str(sample_io: any) -> str:
    if len(sample_io) > 0:
        if type(sample_io[0]) == str:
            return "\n".join(sample_io)
        if type(sample_io[0]) == dict:
            return "\n".join([f"Input:\n{io['input']}\nExpected output:\n{io['output'][0]}" for io in sample_io])
    return sample_io

def humaneval_metric(example, pred, trace=None):
  sample_io_prompt = f"## Sample Test cases: \n{get_sample_io_str(example['sample_io_prompt'])}\n"
  task = example['task']
  algorithm_prompt = ""
  std_input_prompt = ""
  for i in range(0,3):
    plan = getattr(pred, f"plan_{i+1}")
    coding_prompt = coding_agent_prompt(algorithm_prompt, task, plan, sample_io_prompt, std_input_prompt)

    code, _, _ = gpt_chat(coding_prompt)
    code = parse_code(code)
    passed, test_log = humaneval_evaluate_sample_io(example, code, "python")

    if passed:
      return passed
  return False

In [62]:
class GeneratePlans(dspy.Signature):
    """
    Given the task description and the sample input-output pairs, generate a structured plan consisting of three actionable steps (Plan 1, Plan 2, Plan 3) to solve the task effectively. 
    Ensure that each plan outlines specific actions that need to be taken to implement the solution correctly
    """
    task = dspy.InputField(format=str)
    sample_io_prompt = dspy.InputField(format=str)
    plan_1 = dspy.OutputField(format=str)
    plan_2 = dspy.OutputField(format=str)
    plan_3 = dspy.OutputField(format=str)

In [63]:
generatePlans = dspy.Predict(GeneratePlans)

In [64]:
train_data = []
for item in trainset:
  example = dspy.Example(id = item['task_id'],task=item['prompt'], sample_io_prompt = item['sample_io'], plan_1 = "", plan_2 = "", plan_3 = "").with_inputs("task", "sample_io_prompt")
  train_data.append(example)

print(train_data[0])
print(train_data[0]['sample_io_prompt'])
print(train_data[0]['task'])

Example({'id': 'HumanEval/0', 'task': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'sample_io_prompt': ['assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False', 'assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True'], 'plan_1': '', 'plan_2': '', 'plan_3': ''}) (input_keys={'sample_io_prompt', 'task'})
['assert has_close_elements([1.0, 2.0, 3.0], 0.5) == False', 'assert has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3) == True']
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_clo

In [65]:
tp = dspy.MIPROv2(metric=humaneval_metric, auto='light')
optimized = tp.compile(generatePlans, trainset=train_data)


2025/03/06 08:56:21 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 5
valset size: 100

2025/03/06 08:56:23 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/06 08:56:23 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/06 08:56:23 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 12%|█▏        | 4/33 [00:25<03:01,  6.27s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Bootstrapping set 4/5


  9%|▉         | 3/33 [00:23<03:54,  7.82s/it]


...

2025/03/06 09:03:31 INFO dspy.evaluate.evaluate: Average Metric: 23 / 25 (92.0%)
2025/03/06 09:03:31 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 92.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/03/06 09:03:31 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [88.0, 72.0, 84.0, 88.0, 92.0]
2025/03/06 09:03:31 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [86.0]
2025/03/06 09:03:31 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.0
2025/03/06 09:03:31 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/06 09:03:31 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 7 / 8 - Minibatch ==



Average Metric: 21.00 / 25 (84.0%): 100%|██████████| 25/25 [00:37<00:00,  1.52s/it]

2025/03/06 09:04:09 INFO dspy.evaluate.evaluate: Average Metric: 21 / 25 (84.0%)
2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 84.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [88.0, 72.0, 84.0, 88.0, 92.0, 84.0]
2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [86.0]
2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 86.0
2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 8 / 8 - Full Evaluation =====
2025/03/06 09:04:09 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 92.0) from minibatch trials...



Average Metric: 87.00 / 100 (87.0%): 100%|██████████| 100/100 [02:21<00:00,  1.42s/it]

2025/03/06 09:06:31 INFO dspy.evaluate.evaluate: Average Metric: 87 / 100 (87.0%)
2025/03/06 09:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 87.0
2025/03/06 09:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [86.0, 87.0]
2025/03/06 09:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 87.0
2025/03/06 09:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/06 09:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/06 09:06:31 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 87.0!


In [66]:
print(optimized.signature)

StringSignature(task, sample_io_prompt -> plan_1, plan_2, plan_3
    instructions='Given the task description and the sample input-output pairs, generate a structured plan consisting of three actionable steps (Plan 1, Plan 2, Plan 3) to solve the task effectively. Each plan should clearly outline specific actions that need to be taken to implement the solution correctly, including any necessary calculations, logical comparisons, or iterations. Additionally, consider edge cases and potential optimizations to ensure robustness and efficiency in the implementation.'
    task = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'input', 'prefix': 'Task:', 'desc': '${task}'})
    sample_io_prompt = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'input', 'prefix': 'Sample Io Prompt:', 'desc': '${sample_io_prompt}'})
    plan_1 = Field(annotation=str required=True json_schema_extra={'format':

In [67]:

wrapped_text = textwrap.fill(optimized.signature.instructions, width=80)  # 80 可调整
print(wrapped_text)

Given the task description and the sample input-output pairs, generate a
structured plan consisting of three actionable steps (Plan 1, Plan 2, Plan 3) to
solve the task effectively. Each plan should clearly outline specific actions
that need to be taken to implement the solution correctly, including any
necessary calculations, logical comparisons, or iterations. Additionally,
consider edge cases and potential optimizations to ensure robustness and
efficiency in the implementation.


In [32]:
lm.inspect_history(n=1)





[2025-03-06T07:19:04.608689]

System message:

Your input fields are:
1. `task` (str)
2. `sample_io_prompt` (str)

Your output fields are:
1. `plan_1` (str)
2. `plan_2` (str)
3. `plan_3` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## task ## ]]
{task}

[[ ## sample_io_prompt ## ]]
{sample_io_prompt}

[[ ## plan_1 ## ]]
{plan_1}

[[ ## plan_2 ## ]]
{plan_2}

[[ ## plan_3 ## ]]
{plan_3}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `task`, `sample_io_prompt`, produce the fields `plan_1`, `plan_2`, `plan_3`.


User message:

[[ ## task ## ]]

def count_upper(s):
    """
    Given a string s, count the number of uppercase vowels in even indices.
    
    For example:
    count_upper('aBCdEf') returns 1
    count_upper('abcdefg') returns 0
    count_upper('dBBE') returns 0
    """


[[ ## sample_io_prompt ## ]]
[1] «assert count_upper('aBCdEf') == 1»
[2] «assert count_up

In [33]:
optimized.named_predictors()

[('self',
  Predict(StringSignature(task, sample_io_prompt -> plan_1, plan_2, plan_3
      instructions='Given the task description and the sample input-output pairs, generate a structured plan consisting of three actionable steps (Plan 1, Plan 2, Plan 3) to solve the task effectively. Ensure that each plan outlines specific actions that need to be taken to implement the solution correctly.'
      task = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'input', 'prefix': 'Task:', 'desc': '${task}'})
      sample_io_prompt = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'input', 'prefix': 'Sample Io Prompt:', 'desc': '${sample_io_prompt}'})
      plan_1 = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'output', 'prefix': 'Plan 1:', 'desc': '${plan_1}'})
      plan_2 = Field(annotation=str required=True json_schema_extra={'format': 

In [51]:
local_path = r"C:\Users\31646\Desktop\LLM_Project\MapCoder\data\CodeContest\Test.jsonl"
colab_path = "/content/Test.jsonl"

path = local_path

codecontest_ds = read_jsonl(path)
print(codecontest_ds[0].keys())
cc_trainset = random.sample(codecontest_ds, 100)


dict_keys(['name', 'description', 'tags', 'difficulty', 'id', 'sample_io', 'test_list'])


In [54]:
def cc_evaluate_sample_io(
    item: dict,
    cur_imp: str,
    language: str,
):
    return contest_evaluate_public_tests(
        generated_code=cur_imp,
        id=item["id"],
        tests=item["sample_io_prompt"],
        lang=language
  )

def cc_metric(example, pred, trace=None):
  id = example['id']
  sample_io_prompt = f"## Sample Test cases: \n{get_sample_io_str(example['sample_io_prompt'])}\n"
  task = example['task']
  algorithm_prompt = ""
  std_input_prompt = ""
  for i in range(0,3):
    plan = getattr(pred, f"plan_{i+1}")
    coding_prompt = coding_agent_prompt(algorithm_prompt, task, plan, sample_io_prompt, std_input_prompt)

    code, _, _ = gpt_chat(coding_prompt)
    code = parse_code(code)
    passed, test_log = cc_evaluate_sample_io(example, code, "python")

    if passed:
      return passed
  return False

In [55]:
train_data = []
for item in cc_trainset:
  example = dspy.Example(id = item['id'],task=item['description'], sample_io_prompt = item['sample_io'], plan_1 = "", plan_2 = "", plan_3 = "").with_inputs("task", "sample_io_prompt")
  train_data.append(example)

print(train_data[0])
print(train_data[0]['sample_io_prompt'])
print(train_data[0]['task'])

Example({'id': 1603, 'task': 'For two positive integers l and r (l ≤ r) let c(l, r) denote the number of integer pairs (i, j) such that l ≤ i ≤ j ≤ r and \\operatorname{gcd}(i, j) ≥ l. Here, \\operatorname{gcd}(i, j) is the [greatest common divisor (GCD)](https://en.wikipedia.org/wiki/Greatest_common_divisor) of integers i and j.\n\nYouKn0wWho has two integers n and k where 1 ≤ k ≤ n. Let f(n, k) denote the minimum of ∑_{i=1}^{k}{c(x_i+1,x_{i+1})} over all integer sequences 0=x_1 < x_2 < … < x_{k} < x_{k+1}=n.\n\nHelp YouKn0wWho find f(n, k). \n\nInput\n\nThe first line contains a single integer t (1 ≤ t ≤ 3 ⋅ 10^5) — the number of test cases.\n\nThe first and only line of each test case contains two integers n and k (1 ≤ k ≤ n ≤ 10^5). \n\nOutput\n\nFor each test case, print a single integer — f(n, k).\n\nExample\n\nInput\n\n\n4\n6 2\n4 4\n3 1\n10 3\n\n\nOutput\n\n\n8\n4\n6\n11\n\nNote\n\nIn the first test case, YouKn0wWho can select the sequence [0, 2, 6]. So f(6, 2) = c(1, 2) + c(3,

In [56]:
tp = dspy.MIPROv2(metric=cc_metric, auto='light')
optimized = tp.compile(generatePlans, trainset=train_data)

2025/03/06 08:18:57 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: True
num_candidates: 5
valset size: 80

2025/03/06 08:18:58 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/03/06 08:18:58 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/03/06 08:18:58 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=5 sets of demonstrations...


Bootstrapping set 1/5
Bootstrapping set 2/5
Bootstrapping set 3/5


 80%|████████  | 16/20 [05:32<01:23, 20.79s/it]


Bootstrapped 4 full traces after 16 examples for up to 1 rounds, amounting to 16 attempts.
Bootstrapping set 4/5


 30%|███       | 6/20 [01:43<04:01, 17.22s/it]


...

2025/03/06 08:43:24 INFO dspy.evaluate.evaluate: Average Metric: 3 / 25 (12.0%)
2025/03/06 08:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 12.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 4', 'Predictor 0: Few-Shot Set 3'].
2025/03/06 08:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [12.0, 20.0, 36.0, 24.0, 12.0]
2025/03/06 08:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [23.75]
2025/03/06 08:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 23.75
2025/03/06 08:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/06 08:43:24 INFO dspy.teleprompt.mipro_optimizer_v2: == Trial 7 / 8 - Minibatch ==



Average Metric: 8.00 / 25 (32.0%): 100%|██████████| 25/25 [01:56<00:00,  4.64s/it]

2025/03/06 08:45:20 INFO dspy.evaluate.evaluate: Average Metric: 8 / 25 (32.0%)
2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 32.0 on minibatch of size 25 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1'].
2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [12.0, 20.0, 36.0, 24.0, 12.0, 32.0]
2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [23.75]
2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 23.75
2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: ========================================


2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 8 / 8 - Full Evaluation =====
2025/03/06 08:45:20 INFO dspy.teleprompt.mipro_optimizer_v2: Doing full eval on next top averaging program (Avg Score: 36.0) from minibatch trials...



Average Metric: 24.00 / 80 (30.0%): 100%|██████████| 80/80 [05:04<00:00,  3.80s/it]

2025/03/06 08:50:25 INFO dspy.evaluate.evaluate: Average Metric: 24 / 80 (30.0%)
2025/03/06 08:50:25 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 30.0
2025/03/06 08:50:25 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [23.75, 30.0]
2025/03/06 08:50:25 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 30.0
2025/03/06 08:50:25 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/03/06 08:50:25 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/03/06 08:50:25 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 30.0!


In [ ]:

wrapped_text = textwrap.fill(optimized.signature.instructions, width=80)  # 80 可调整
print(wrapped_text)

Given the task of restoring an original array `a` from a sorted sums array `b`,
analyze the provided input and output to devise structured plans. Your task is
to outline three distinct plans that will guide the implementation of the
solution. Each plan should break down the steps necessary to achieve the goal,
considering edge cases and ensuring efficiency. Utilize the sample input/output
provided to inform your plans and ensure clarity in your approach.


In [58]:
optimized.named_predictors()

[('self',
  Predict(StringSignature(task, sample_io_prompt -> plan_1, plan_2, plan_3
      instructions='Given the task of restoring an original array `a` from a sorted sums array `b`, analyze the provided input and output to devise structured plans. Your task is to outline three distinct plans that will guide the implementation of the solution. Each plan should break down the steps necessary to achieve the goal, considering edge cases and ensuring efficiency. Utilize the sample input/output provided to inform your plans and ensure clarity in your approach.'
      task = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'input', 'prefix': 'Task:', 'desc': '${task}'})
      sample_io_prompt = Field(annotation=str required=True json_schema_extra={'format': <class 'str'>, '__dspy_field_type': 'input', 'prefix': 'Sample Io Prompt:', 'desc': '${sample_io_prompt}'})
      plan_1 = Field(annotation=str required=True json_schema_extra={'format'